In [5]:
%pip install xgboost


  Using cached xgboost-2.1.3-py3-none-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 330.3 kB/s eta 0:06:19
   ---------------------------------------- 0.1/124.9 MB 409.6 kB/s eta 0:05:05
   ---------------------------------------- 0.1/124.9 MB 547.6 kB/s eta 0:03:48
   ---------------------------------------- 0.2/124.9 MB 787.7 kB/s eta 0:02:39
   ---------------------------------------- 0.3/124.9 MB 930.9 kB/s eta 0:02:14
   ---------------------------------------- 0.3/124.9 MB 1.0 MB/s eta 0:02:03
   ---------------------------------------- 0.4/124.9 MB 1.1 MB/s eta 0:01:49
   ---------------------------------------- 0.6/124.9 MB 1.4 MB/s eta 0:01:30
   ---------------------------------------- 0.7/124.9 MB 1.5 MB/s eta 0:01:22
   -

In [7]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
df = pd.read_csv(r"C:\Users\Sheethal Rajeev\Downloads\airline\Airline Dataset.csv")  # Replace 'your_dataset.csv' with the actual path to your file

# Data Cleaning: Remove null values if any
df.dropna(inplace=True)

# Feature Engineering: Extract day of the week, month from Departure Date
df['Departure Date'] = pd.to_datetime(df['Departure Date'])
df['Day_of_Week'] = df['Departure Date'].dt.dayofweek
df['Month'] = df['Departure Date'].dt.month

In [37]:
# Prepare the features
features = ['Airport Name', 'Airport Country Code', 'Country Name', 'Airport Continent', 'Day_of_Week', 'Month', 'Pilot Name']
X = df[features].copy()  # Create a copy to avoid SettingWithCopyWarning

In [39]:
# Label Encoding for categorical features
label_encoders = {}
for feature in ['Airport Name', 'Airport Country Code', 'Country Name', 'Airport Continent', 'Pilot Name']:
    le = LabelEncoder()
    X.loc[:, feature] = le.fit_transform(X[feature])  # Use .loc to avoid SettingWithCopyWarning
    label_encoders[feature] = le

In [43]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Prepare the target variable
y = df['Flight Status'].apply(lambda x: 1 if x == 'Delayed' else 0)  # Adjust this based on your actual data

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [45]:
# Initialize and train the XGBoost classifier
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train, y_train)

# Make predictions
y_pred = xgb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.6539748529709998
              precision    recall  f1-score   support

           0       0.66      0.97      0.79     13069
           1       0.33      0.02      0.05      6655

    accuracy                           0.65     19724
   macro avg       0.50      0.50      0.42     19724
weighted avg       0.55      0.65      0.54     19724

